In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import re
from selenium.webdriver.common.by import By
from datetime import datetime 

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromeDriver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

C:\Users\wjstj\AppData\Local\Temp\ipykernel_3064\1343117472.py:5: DeprecationWarning: use options instead of chrome_options
  chromeDriver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


In [6]:
chromeDriver.get("http://www.statiz.co.kr/schedule.php")
html = chromeDriver.page_source
bsObject = BeautifulSoup(html, 'html.parser')

# 날짜정보 받아오기
now = datetime.now().strftime("%Y-%m-%d")
now = '2023-04-09'

init_url = str(bsObject.find_all('table', {'class' : 'table table-striped table-bordered'})).split(f'boxscore.php?date={now}&amp;')[-5:]

In [101]:
def transform_pitching_data(pitching_list,team):
    player_list = []
    for i in pitching_list:
        date = now
        name = i.split('>')[1].split('<')[0]
        birth = i.split('"')[0]
        today_type = i.split('(')[1].split(',')[0]
        if '"popup/pitlog.php' in today_type:
            today_type = '' 
        IP = i.split('<b>')[1].split('</b>')[0]
        TBF = i.split('<td>')[1].split('</td>')[0]
        H = i.split('<td>')[2].split('</td>')[0]
        R = i.split('<td>')[3].split('</td>')[0]
        ER = i.split('<td>')[4].split('</td>')[0]
        BB = i.split('<td>')[5].split('</td>')[0]
        HBP = i.split('<td>')[6].split('</td>')[0]
        K = i.split('<td>')[7].split('</td>')[0]
        HR = i.split('<td>')[8].split('</td>')[0]
        GO_FO = i.split('<td align="center">')[2].split('</td>')[0]
        PIT_S = i.split('<td align="center">')[3].split('</td>')[0]
        IR_IS = i.split('<td align="center">')[4].split('</td>')[0]
        GSC = i.split('<td>')[9].split('</td>')[0]
        ERA = i.split('<td align="center">')[5].split('</td>')[0]
        WHIP = i.split('<td align="center">')[6].split('</td>')[0]
        LI = i.split('<td>')[10].split('</td>')[0]
        WPA = i.split('<td>')[11].split('</td>')[0]
        RE24 = i.split('<td>')[12].split('</td>')[0]
        player_list.append([date, name,birth,team,today_type,IP,TBF,H,R,ER,BB,HBP,K,HR,GO_FO,PIT_S,IR_IS,GSC,ERA,WHIP,LI,WPA,RE24])
    return player_list

def transform_batting_data(batting_list, team):
    player_list = []
    for i in batting_list:
        date = now
        name = i.split('>')[1].split('<')[0]
        birth = i.split('"')[0]
        P = i.split('<td>')[1].split('</td>')[0]
        TPA = i.split('<b>')[1].split('</b>')[0]
        AB = i.split('<td>')[3].split('</td>')[0]
        R = i.split('<td>')[4].split('</td>')[0]
        H = i.split('<td>')[5].split('</td>')[0]
        HR = i.split('<td>')[6].split('</td>')[0]
        RBI = i.split('<td>')[7].split('</td>')[0]
        BB = i.split('<td>')[8].split('</td>')[0]
        HBP = i.split('<td>')[9].split('</td>')[0]   
        SO = i.split('<td>')[10].split('</td>')[0]
        GO = i.split('<td>')[11].split('</td>')[0]
        FO = i.split('<td>')[12].split('</td>')[0]
        PIT = i.split('<td>')[13].split('</td>')[0]
        GDP = i.split('<td>')[14].split('</td>')[0]
        LOB = i.split('<td>')[15].split('</td>')[0]
        AVG = i.split('<td>')[16].split('</td>')[0]
        OPS = i.split('<td>')[17].split('</td>')[0]
        LI = i.split('<td>')[18].split('</td>')[0]
        WPA = i.split('<td>')[19].split('</td>')[0]
        RE24 = i.split('<td>')[20].split('</td>')[0]
        
        player_list.append([date, name,birth,team,P,TPA,AB,R,H,HR,RBI,BB,HBP,SO,GO,FO,PIT,GDP,LOB,AVG,OPS,LI,WPA,RE24])
    return player_list

In [106]:
batting = pd.DataFrame()
pitching = pd.DataFrame()
# 날짜 기준으로 해당 경기 데이터 순서대로 크롤링
for i in init_url:
    stadium = str(i).split('>')[0]
    
    # 경기 url만들기 
    url = f'http://www.statiz.co.kr/boxscore.php?opt=4&date={now}&{stadium}'
    chromeDriver.get(url)
    html = chromeDriver.page_source
    bsObject = BeautifulSoup(html, 'html.parser')
    
    
    # 홈팀
    home_batting = str(bsObject.find_all('h3')[1])
    hometeam = home_batting.split('(')[1].split(')')[0]
    
    # 원정팀 
    away_batting = str(bsObject.find_all('h3')[2])
    awayteam = away_batting.split('(')[1].split(')')[0]

    print(f'{hometeam} vs {awayteam} 경기 데이터 크롤링 !')
    
    # 타자 컬럼 데이터 추출
    batting_columns = ['date', 'name', 'birth', 'team']
    for i in bsObject.find_all('th')[2:22]:
        batting_columns.append(str(i).split('<th>')[1].split('</th>')[0])
    
    ############################
    ## 홈팀 데이터 추출 진행 ! ##  
    ############################

    # 홈팀 타자들의 정보 리스트로 담아두기
    home_batting_list = []
    for i in str(bsObject.find_all('table')[3]).split('birth=')[1:]:
        home_batting_list.append(i)

    # 타자별 정보 분리 및 2차월 배열로 저장           
    home_player_list = transform_batting_data(home_batting_list, hometeam)
    home_team_batting = pd.DataFrame(home_player_list,columns=batting_columns)
    

    ##############################
    ## 원정팀 데이터 추출 진행 ! ##  
    ##############################
    
    # 원정팀 타자들의 정보 리스트로 담아두기
    away_batting_list = []
    for i in str(bsObject.find_all('table')[5]).split('birth=')[1:]:
        away_batting_list.append(i)
    
    # 타자별 정보 분리 및 2차월 배열로 저장
    away_player_list = transform_batting_data(away_batting_list, awayteam)
    away_team_batting = pd.DataFrame(away_player_list,columns=batting_columns)
    

    # 홈팀, 원정팀 정보 concat
    batting = pd.concat([batting,home_team_batting,away_team_batting])

    ##############################
    ## 타자 정보가져오기 완료 !! ##
    ##############################

    # 타자 컬럼 데이터 추출
    pitching_columns = ['date', 'name', 'birth', 'team','today_type']
    for i in str(bsObject.find_all('table')[7]).split('<th>')[2:]:
        
        # 주석처리된 지표 핸들링
        if i.split('</th>')[0] != 'RS':
            pitching_columns.append(i.split('</th>')[0])

    ############################
    ## 홈팀 데이터 추출 진행 ! ##  
    ############################

    # 홈팀 투수들의 정보 리스트로 담아두기
    home_pitching_list = []
    for i in str(bsObject.find_all('table')[7]).split('birth=')[1:]:
        home_pitching_list.append(i)

    # 투수별 정보 분리 및 2차월 배열로 저장
    player_list = transform_pitching_data(home_pitching_list, hometeam)
    home_team_pitching = pd.DataFrame(player_list,columns=pitching_columns)

    ##############################
    ## 원정팀 데이터 추출 진행 ! ##  
    ##############################

    # 원정팀 투수들의 정보 리스트로 담아두기
    away_pitching_list = []
    for i in str(bsObject.find_all('table')[9]).split('birth=')[1:]:
        away_pitching_list.append(i)

    # 투수별 정보 분리 및 2차월 배열로 저장
    player_list = transform_pitching_data(away_pitching_list, awayteam)
    away_team_pitching = pd.DataFrame(player_list,columns=pitching_columns)

    # 홈팀, 원정팀 정보 concat
    pitching = pd.concat([pitching,home_team_pitching,away_team_pitching])



SSG vs 한화 경기 데이터 크롤링 !
KT vs 롯데 경기 데이터 크롤링 !
삼성 vs LG 경기 데이터 크롤링 !
키움 vs NC 경기 데이터 크롤링 !
두산 vs KIA 경기 데이터 크롤링 !


In [107]:
pitching

,date,name,birth,team,today_type,IP,TBF,H,R,ER,...,HR,GO-FO,PIT-S,IR-IS,GSC,ERA,WHIP,LI,WPA,RE24
0,2023-04-09,맥카티,1995-10-12,SSG,승,6.0,20,1,0,0,...,0,7-6,93-63,0-0,72,7.71,1.71,1.0,0.333,3.33
1,2023-04-09,최민준,1999-06-11,SSG,홀,1.0,5,2,0,0,...,0,2-1,21-13,0-0,,0.00,1.00,2.1,0.055,0.56
2,2023-04-09,고효준,1983-02-08,SSG,홀,1.0,4,0,0,0,...,0,0-3,21-15,0-0,,0.00,0.30,1.0,0.051,0.55
3,2023-04-09,노경은,1984-03-11,SSG,세,1.0,5,0,0,0,...,0,0-0,24-11,0-0,,6.75,3.00,2.1,0.041,0.55
0,2023-04-09,페냐,1990-02-25,한화,패,5.0,23,4,3,2,...,0,7-7,95-51,0-0,48,7.00,1.67,0.9,-0.088,-0.23
1,2023-04-09,김기중,2002-11-16,한화,,2.0,6,0,0,0,...,0,1-3,23-17,0-0,,0.00,0.00,0.3,0.045,1.11
2,2023-04-09,윤대경,1994-04-09,한화,,1.0,4,1,0,0,...,0,0-2,14-9,0-0,,0.00,1.00,0.2,0.015,0.55
3,2023-04-09,한승혁,1993-01-03,한화,,1.0,3,0,0,0,...,0,2-1,3-3,0-0,,13.50,1.80,0.1,0.008,0.55
0,2023-04-09,배제성,1996-09-29,KT,,6.0,25,5,0,0,...,0,5-6,98-48,0-0,61,0.00,1.33,1.3,0.355,3.33
1,2023-04-09,박세진,1997-06-27,KT,패,0.0,4,4,3,3,...,0,0-0,12-7,0-0,,20.25,6.75,1.8,-0.357,-3.35


In [108]:
batting

,date,name,birth,team,P,TPA,AB,R,H,HR,...,GO,FO,PIT,GDP,LOB,AVG,OPS,LI,WPA,RE24
0,2023-04-09,추신수,1982-07-13,SSG,9,4,4,0,0,0,...,2,1,15,0,1,0.167,0.816,0.65,-0.068,-0.95
1,2023-04-09,오태곤,1991-11-18,SSG,9,0,0,0,0,0,...,0,0,0,0,0,0.308,0.708,,0.000,0.00
2,2023-04-09,최지훈,1997-07-23,SSG,8,4,4,2,2,0,...,0,2,13,0,0,0.240,0.665,0.55,0.072,0.74
3,2023-04-09,최정,1987-02-28,SSG,5,4,4,0,1,0,...,1,2,20,0,1,0.217,0.731,0.81,-0.004,-0.14
4,2023-04-09,최주환,1988-02-28,SSG,4,4,3,0,0,0,...,0,2,15,0,1,0.263,0.702,0.59,-0.020,-0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,2023-04-09,주효상,1997-11-11,KIA,2,3,2,1,0,0,...,1,1,13,0,0,0.000,0.167,1.63,0.079,0.39
11,2023-04-09,최정용,1996-10-24,KIA,r,0,0,0,0,0,...,0,0,0,0,0,0.000,0.000,,0.000,0.00
12,2023-04-09,한승택,1994-06-21,KIA,2,1,1,0,0,0,...,1,0,3,0,3,0.000,0.222,7.87,-0.198,-0.81
13,2023-04-09,김규성,1997-03-08,KIA,4,3,2,0,0,0,...,0,0,13,0,2,0.333,0.788,2.28,-0.121,-0.91
